[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google/vizier/blob/main/docs/guides/user/running_vizier.ipynb)

# Vizier Basics
Below, we provide examples of how to:

*   Define a problem statement and study configuration.
*   Start a client.
*   (Optionally) Connect the client to a server.
*   Perform a typical tuning loop.
*   Use other client APIs.

## Installation and reference imports

In [ ]:
%pip install 'google-vizier[jax]'

In [4]:
from vizier import service
from vizier.service import clients
from vizier.service import pyvizier as vz
from vizier.service import servers

## Setting up the problem statement
Here we setup the problem statement, which contains information about the search space and the metrics to optimize.

In [5]:
problem = vz.ProblemStatement()
problem.search_space.root.add_float_param('x', 0.0, 1.0)
problem.search_space.root.add_float_param('y', 0.0, 1.0)
problem.metric_information.append(
    vz.MetricInformation(
        name='maximize_metric', goal=vz.ObjectiveMetricGoal.MAXIMIZE))


def evaluate(x: float, y: float) -> float:
  return x**2 - y**2

## Setting up the study configuration
The study configuration contains additional information, such as the algorithm to use and level of noise that we think the objective will have.

In [6]:
study_config = vz.StudyConfig.from_problem(problem)
study_config.algorithm = 'DEFAULT'

## Setting up the client
Starts a `study_client`, which can be either in **local mode (default)** or **distributed mode.**

**Local Mode:** The client has no `endpoint` set, and will implicitly create a local Vizier Service which will be shared across other clients in the same Python process. Studies will then be stored locally in a SQL database file located at `service.VIZIER_DB_PATH`.

In [7]:
study_client = clients.Study.from_study_config(study_config, owner='owner', study_id='example_study_id')
print('Local SQL database file located at: ', service.VIZIER_DB_PATH)

Local SQL database file located at:  /home/awkwabear/anaconda3/envs/bayop/lib/python3.10/site-packages/vizier/_src/service/vizier.db


**Distributed mode:** The service may be explicitly created, wrapped as a server in a separate process to accept requests from all other client processses. Details such as the `database_url`, `port`, `policy_factory`, etc. can be configured in the server's initializer.

All client processes (on a single machine or over multiple machines) will connect to this server via a globally specified `endpoint`.

In [8]:
server = servers.DefaultVizierServer()  # Ideally created on a separate process such as a server machine.
clients.environment_variables.server_endpoint = server.endpoint  # Server address.
study_client = clients.Study.from_study_config(study_config, owner='owner', study_id = 'example_study_id')  # Now connects to the explicitly created server.

## Client Parallelization
Regardless of whether the setup is local or distributed, we may simultaneously create multiple clients to work on the same study, useful for parallelizing evaluation workload.

In [9]:
another_study_client = clients.Study.from_resource_name(study_client.resource_name)

## Obtaining suggestions
Start requesting suggestions from the server, for evaluating objectives. Suggestions can be made sequentially (`count=1`) or in batches (`count>1`).

In [10]:
for i in range(10):
  suggestions = study_client.suggest(count=1)
  for suggestion in suggestions:
    x = suggestion.parameters['x']
    y = suggestion.parameters['y']
    objective = evaluate(x, y)
    print(f'Iteration {i}, suggestion ({x},{y}) led to objective value {objective}.')
    final_measurement = vz.Measurement({'maximize_metric': objective})
    suggestion.complete(final_measurement)

2024-11-20 17:10:47.412004: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732158647.421837   94591 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732158647.424866   94591 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Iteration 0, suggestion (0.5,0.5) led to objective value 0.0.
Iteration 1, suggestion (0.28000000000000086,0.71942713626663) led to objective value -0.4391754043968038.
Iteration 2, suggestion (0.7399999999999984,0.7396063871241385) led to objective value 0.000582392125176634.
Iteration 3, suggestion (0.9999999999999983,0.48014892912893015) led to objective value 0.7694570058563382.
Iteration 4, suggestion (0.9999905103930361,0.2001489291289329) led to objective value 0.9599214270446662.
Iteration 5, suggestion (1.0,0.0) led to objective value 1.0.
Iteration 6, suggestion (1.0,6.146285532821177e-15) led to objective value 1.0.
Iteration 7, suggestion (0.6681539118469481,0.0) led to objective value 0.44642964991637923.
Iteration 8, suggestion (1.0,0.05073318898474313) led to objective value 0.9974261435354383.
Iteration 9, suggestion (0.9455716267534012,0.0) led to objective value 0.8941057013210735.


## Find optimal trial
Find the best objective so far, with corresponding suggestion value. For multiobjective cases, there may be multiple outputs of `optimal_trials()`, all corresponding to a Pareto-optimal curve.

In [11]:
for optimal_trial in study_client.optimal_trials():
  optimal_trial = optimal_trial.materialize()
  print("Optimal Trial Suggestion and Objective:", optimal_trial.parameters,
        optimal_trial.final_measurement)

Optimal Trial Suggestion and Objective: ParameterDict(_items={'x': 1.0, 'y': 0.0}) Measurement(metrics={'maximize_metric': Metric(value=1.0, std=None)}, elapsed_secs=0.0, steps=0, checkpoint_path='')
Optimal Trial Suggestion and Objective: ParameterDict(_items={'x': 1.0, 'y': 6.146285532821177e-15}) Measurement(metrics={'maximize_metric': Metric(value=1.0, std=None)}, elapsed_secs=0.0, steps=0, checkpoint_path='')


## Other client commands
The `study_client` can also send other requests, such as the following:

In [12]:
study_client.get_trial(1)  # Get the first trial.
study_client.trials()  # Get all trials so far.

# Obtain only the completed trials.
trial_filter = vz.TrialFilter(status=[vz.TrialStatus.COMPLETED])
study_client.trials(trial_filter=trial_filter)

TrialIterable(_iterable_factory=<function Study.trials.<locals>.iterable_factory at 0x7dce7441d990>, _client=VizierClient(_study_resource_name='owners/owner/studies/example_study_id', _client_id='unused_client_id'))